In [ ]:
pip install git-lfs

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


In [117]:
repository = "naver-clova-ix/donut-base-finetuned-docvqa"
model_id=repository.split("/")[-1]
s3_location=f"s3://{sess.default_bucket()}/custom_inference/{model_id}/model.tar.gz"

In [ ]:
!git lfs install
!git clone https://huggingface.co/$repository

In [120]:
!cp -r code/ $model_id/code/

In [ ]:
%cd $model_id
!tar zcvf model.tar.gz *

In [ ]:
!aws s3 cp model.tar.gz $s3_location

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.serializers import DataSerializer

# create a serializer for the data
#image_serializer = DataSerializer(content_type='image/x-image')

# YOU WILL WANT TO DOUBLE CHECK YOU ARE USING THE LATEST VERSIONS OF PYTORCH AND TRANSFORMERS 

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_location,       # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint     # pytorch version used
   transformers_version='4.26.0',
   pytorch_version='1.13.1',
   py_version='py39'# python version used
)

# deploy the endpoint endpoint
# make sure this instance type is the correct instance you want to deploy. for testing purposes a ml.m5.xlarge is sufficient
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge"
    )

In [126]:
# change the receipt.JPG to your JPG filename
import base64
with open('./receipt.JPG',"rb") as image_file:
          encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

In [127]:
test_data = { 'inputs': {'image': encoded_string }, 'question' : 'what is the rounded total?'}

In [ ]:
predictor.predict(test_data)